In [2]:
from datetime import datetime, timedelta
from analyze_stock import *  
from scrape_url import *

# Call the function and store the result
day = '2023-12-28'

# Set the time frame
date = datetime.strptime(day, '%Y-%m-%d')
end_date = date + timedelta(days=7)
start_date = date - timedelta(days=14)

# Check if start_date is earlier than today, if not change start_date to today - 7 days
current_date = datetime.now()
start_date = current_date - timedelta(days=14) if start_date >= current_date else start_date

# First page
url = 'https://finance.yahoo.com/calendar/earnings/?day=' + day
print(url)

ticker_data_list = process_earnings_table(extract_table(url))
    
# Second page
url += '&offset=100&size=100'

ticker_data_list = process_earnings_table(extract_table(url), ticker_data_list)

# Pandas Dataframe
ticker_data_sorted = convert_to_dataframe(ticker_data_list)

filtered_tickers = []

# Iterate through the sorted list of tickers
for index, row in ticker_data_sorted.iterrows():
    ticker = row['Symbol']
    
    if ticker in filtered_tickers:
        continue

    # Use the function from utils.py to analyze stock options
    options_metrics = analyze_stock_options(ticker)

    if pd.isna(options_metrics['avg_put_implied_volatility']):
        continue
    
    if options_metrics['total_itm_calls'] == 0 and options_metrics['total_itm_puts'] == 0:
        continue

    # Print 
    print_options_data(ticker, options_metrics)
    
    # Filtered tickers
    filtered_tickers.append(ticker)
    
del filtered_tickers, ticker_data_list, ticker_data_sorted

https://finance.yahoo.com/calendar/earnings/?day=2023-12-28


/Users/saumikdana/Machine_Learning_Trading/data/scrape_url.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/Users/saumikdana/Machine_Learning_Trading/data/scrape_url.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ticker_data = pd.concat(ticker_data_list, ignore_index=True)


Options data for OMEX:
Average IV for Calls: 1.6675969924316403
Average IV for Puts: 1.6999944362386068
Total Call Volume: 676
Total Call open interest: 5622
Total Call engagement: 6298
Total Put Volume: 98.0
Total Put open interest: 1737
Total Put engagement: 1835.0
Number of ITM Call Options: 10
Number of ITM Put Options: 1
Options data for LILM:
Average IV for Calls: 1.7971221997750948
Average IV for Puts: 2.213900289950284
Total Call Volume: 498.0
Total Call open interest: 43263
Total Call engagement: 43761.0
Total Put Volume: 620.0
Total Put open interest: 4703
Total Put engagement: 5323.0
Number of ITM Call Options: 5
Number of ITM Put Options: 16
Options data for ARVL:
Average IV for Calls: 1.906865809830729
Average IV for Puts: 2.4918372261439727
Total Call Volume: 1297.0
Total Call open interest: 31430
Total Call engagement: 32727.0
Total Put Volume: 1389.0
Total Put open interest: 10929
Total Put engagement: 12318.0
Number of ITM Call Options: 4
Number of ITM Put Options: 19
